# 🧠 AI/ML Tool Template

This notebook demonstrates how to build AI-powered tools using:
- OpenAI API (GPT models)
- Anthropic API (Claude models)
- Prompt engineering techniques
- Response processing

Choose your preferred AI provider and customize for your needs!

## 1. Setup

In [ ]:
import sys
from pathlib import Path

# Add src to path
sys.path.insert(0, str(Path.cwd().parent))

from src.config import settings, check_api_keys
from src.utils import get_timestamp, Timer

print("✅ Setup complete!")
print(f"Timestamp: {get_timestamp()}")
print()

# Check which API keys are configured
check_api_keys()

## 2. OpenAI API Example

Using GPT models from OpenAI.

In [ ]:
# Check if OpenAI API key is configured
if not settings.OPENAI_API_KEY:
    print("⚠️  OpenAI API key not configured")
    print("   1. Get API key from https://platform.openai.com/api-keys")
    print("   2. Add to .env: OPENAI_API_KEY=sk-...")
else:
    print("✅ OpenAI API key found")
    
    try:
        from openai import OpenAI
        
        # Initialize client
        client = OpenAI(api_key=settings.OPENAI_API_KEY)
        
        print("✅ OpenAI client initialized")
        
    except ImportError:
        print("❌ OpenAI package not installed")
        print("   Run: pip install openai")

In [ ]:
# Example 1: Simple completion
def ask_gpt(prompt: str, model: str = "gpt-4") -> str:
    """Send prompt to GPT and get response."""
    
    response = client.chat.completions.create(
        model=model,
        messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": prompt}
        ],
        temperature=0.7,
        max_tokens=500
    )
    
    return response.choices[0].message.content

# Test it (uncomment if API key is configured)
if settings.OPENAI_API_KEY:
    with Timer("OpenAI API call"):
        result = ask_gpt("Explain Python list comprehensions in one sentence.")
        print("\n📝 Response:")
        print(result)

## 3. Anthropic Claude API Example

Using Claude models from Anthropic.

In [ ]:
# Check if Anthropic API key is configured
if not settings.ANTHROPIC_API_KEY:
    print("⚠️  Anthropic API key not configured")
    print("   1. Get API key from https://console.anthropic.com/")
    print("   2. Add to .env: ANTHROPIC_API_KEY=sk-ant-...")
else:
    print("✅ Anthropic API key found")
    
    try:
        from anthropic import Anthropic
        
        # Initialize client
        anthropic = Anthropic(api_key=settings.ANTHROPIC_API_KEY)
        
        print("✅ Anthropic client initialized")
        
    except ImportError:
        print("❌ Anthropic package not installed")
        print("   Run: pip install anthropic")

In [ ]:
# Example 2: Ask Claude
def ask_claude(prompt: str, model: str = "claude-3-sonnet-20240229") -> str:
    """Send prompt to Claude and get response."""
    
    message = anthropic.messages.create(
        model=model,
        max_tokens=500,
        messages=[
            {"role": "user", "content": prompt}
        ]
    )
    
    return message.content[0].text

# Test it (uncomment if API key is configured)
if settings.ANTHROPIC_API_KEY:
    with Timer("Claude API call"):
        result = ask_claude("Explain Python list comprehensions in one sentence.")
        print("\n📝 Response:")
        print(result)

## 4. Practical Example: Code Explainer

In [ ]:
def explain_code(code: str, use_claude: bool = False) -> str:
    """
    Explain what a piece of code does.
    
    Args:
        code: Code snippet to explain
        use_claude: Use Claude instead of GPT
    
    Returns:
        Explanation of the code
    """
    prompt = f"""Explain what this Python code does in simple terms:

```python
{code}
```

Provide:
1. A brief summary
2. What each line does
3. The output/result
"""
    
    if use_claude and settings.ANTHROPIC_API_KEY:
        return ask_claude(prompt)
    elif settings.OPENAI_API_KEY:
        return ask_gpt(prompt)
    else:
        return "❌ No API key configured"

# Example code to explain
example_code = """
numbers = [1, 2, 3, 4, 5]
squares = [x**2 for x in numbers if x % 2 == 0]
print(squares)
"""

print("Code to explain:")
print(example_code)
print("\n" + "="*60 + "\n")

# Get explanation (choose your preferred AI)
if settings.OPENAI_API_KEY or settings.ANTHROPIC_API_KEY:
    with Timer("Code explanation"):
        explanation = explain_code(example_code, use_claude=bool(settings.ANTHROPIC_API_KEY))
        print("Explanation:")
        print(explanation)
else:
    print("⚠️  Please configure an API key to use this feature")

## 5. Practical Example: Text Summarizer

In [ ]:
def summarize_text(text: str, max_sentences: int = 3, use_claude: bool = False) -> str:
    """
    Summarize a long text.
    
    Args:
        text: Text to summarize
        max_sentences: Maximum sentences in summary
        use_claude: Use Claude instead of GPT
    
    Returns:
        Summarized text
    """
    prompt = f"""Summarize the following text in {max_sentences} sentences or less:

{text}

Summary:"""
    
    if use_claude and settings.ANTHROPIC_API_KEY:
        return ask_claude(prompt)
    elif settings.OPENAI_API_KEY:
        return ask_gpt(prompt)
    else:
        return "❌ No API key configured"

# Example long text
long_text = """
Python is a high-level, interpreted programming language known for its simplicity 
and readability. Created by Guido van Rossum and first released in 1991, Python 
emphasizes code readability with its notable use of significant indentation. 
Its language constructs and object-oriented approach aim to help programmers 
write clear, logical code for small and large-scale projects. Python is dynamically 
typed and garbage-collected. It supports multiple programming paradigms, including 
structured, object-oriented, and functional programming. Python is often described 
as a "batteries included" language due to its comprehensive standard library.
"""

print("Original text:")
print(long_text)
print(f"\nLength: {len(long_text)} characters")
print("\n" + "="*60 + "\n")

if settings.OPENAI_API_KEY or settings.ANTHROPIC_API_KEY:
    with Timer("Summarization"):
        summary = summarize_text(long_text, use_claude=bool(settings.ANTHROPIC_API_KEY))
        print("Summary:")
        print(summary)
        print(f"\nSummary length: {len(summary)} characters")
else:
    print("⚠️  Please configure an API key to use this feature")

## 6. Practical Example: Q&A System

In [ ]:
def qa_system(context: str, question: str, use_claude: bool = False) -> str:
    """
    Answer questions based on provided context.
    
    Args:
        context: Background information
        question: Question to answer
        use_claude: Use Claude instead of GPT
    
    Returns:
        Answer to the question
    """
    prompt = f"""Context:
{context}

Question: {question}

Answer the question based only on the context provided above. If the answer 
cannot be found in the context, say "I don't have enough information to answer that."

Answer:"""
    
    if use_claude and settings.ANTHROPIC_API_KEY:
        return ask_claude(prompt)
    elif settings.OPENAI_API_KEY:
        return ask_gpt(prompt)
    else:
        return "❌ No API key configured"

# Example context and questions
context = """
The Python Tool Starter Kit is a repository designed for students and developers 
who want to quickly build Python tools. It includes templates for data processing, 
web applications, bots, and AI/ML tools. The kit comes with pre-configured 
environment management, setup scripts, and best practices. It supports integration 
with OpenAI and Anthropic APIs for AI-powered applications.
"""

questions = [
    "What is the Python Tool Starter Kit?",
    "What types of templates are included?",
    "Which AI APIs does it support?",
]

print("Context:")
print(context)
print("\n" + "="*60 + "\n")

if settings.OPENAI_API_KEY or settings.ANTHROPIC_API_KEY:
    for i, question in enumerate(questions, 1):
        print(f"Q{i}: {question}")
        
        with Timer(f"Answer {i}"):
            answer = qa_system(context, question, use_claude=bool(settings.ANTHROPIC_API_KEY))
            print(f"A{i}: {answer}")
        
        print("\n" + "-"*60 + "\n")
else:
    print("⚠️  Please configure an API key to use this feature")

## 7. Advanced: Conversation History

In [ ]:
class ChatBot:
    """Simple chatbot with conversation history."""
    
    def __init__(self, use_claude: bool = False):
        self.history = []
        self.use_claude = use_claude
        
        # System message
        self.system_message = "You are a helpful Python programming assistant."
    
    def chat(self, user_message: str) -> str:
        """Send message and get response."""
        
        # Add user message to history
        self.history.append({"role": "user", "content": user_message})
        
        # Get response based on API choice
        if self.use_claude and settings.ANTHROPIC_API_KEY:
            response = anthropic.messages.create(
                model="claude-3-sonnet-20240229",
                max_tokens=500,
                system=self.system_message,
                messages=self.history
            )
            assistant_message = response.content[0].text
            
        elif settings.OPENAI_API_KEY:
            messages = [{"role": "system", "content": self.system_message}] + self.history
            response = client.chat.completions.create(
                model="gpt-4",
                messages=messages,
                max_tokens=500
            )
            assistant_message = response.choices[0].message.content
        else:
            return "❌ No API key configured"
        
        # Add assistant response to history
        self.history.append({"role": "assistant", "content": assistant_message})
        
        return assistant_message
    
    def clear_history(self):
        """Clear conversation history."""
        self.history = []

# Create chatbot
if settings.OPENAI_API_KEY or settings.ANTHROPIC_API_KEY:
    bot = ChatBot(use_claude=bool(settings.ANTHROPIC_API_KEY))

    # Have a conversation
    conversation = [
        "What is a Python list?",
        "How do I add items to it?",
        "What's the difference between append and extend?"
    ]

    for message in conversation:
        print(f"You: {message}")
        
        with Timer("Response"):
            response = bot.chat(message)
            print(f"Bot: {response}")
        
        print("\n" + "-"*60 + "\n")
else:
    print("⚠️  Please configure an API key to use this feature")

## 💡 Next Steps

1. **Get API keys** from OpenAI or Anthropic
2. **Add to .env** file
3. **Run examples** above
4. **Customize prompts** for your use case
5. **Build your AI tool** using these patterns

### Ideas for AI Tools:
- 📝 Content generator
- 🔍 Document Q&A system
- 💻 Code reviewer/helper
- 📧 Email responder
- 🎓 Learning assistant
- 📊 Data analyzer

Happy building! 🚀